In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb
import seaborn as sns

In [2]:
train = pd.read_csv("./data/train.csv", encoding="utf-8")
test = pd.read_csv("./data/test.csv", encoding="utf-8")

le = LabelEncoder()
train['IsHoliday'] = le.fit_transform(train['IsHoliday'])
test['IsHoliday'] = le.fit_transform(test['IsHoliday'])

train[['day', 'month', 'year']] = train['Date'].str.split('/', expand=True).astype(int)
test[['day', 'month', 'year']] = test['Date'].str.split('/', expand=True).astype(int)

test

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,day,month,year
0,1,1,05/10/2012,68.55,3.617,8077.89,NaN,18.22,3617.43,3626.14,6.573,0,5,10,2012
1,2,1,12/10/2012,62.99,3.601,2086.18,NaN,8.11,602.36,5926.45,6.573,0,12,10,2012
2,3,1,19/10/2012,67.97,3.594,950.33,NaN,4.93,80.25,2312.85,6.573,0,19,10,2012
3,4,1,26/10/2012,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,0,26,10,2012
4,5,2,05/10/2012,70.27,3.617,6037.76,NaN,10.04,3027.37,3853.40,6.170,0,5,10,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,176,44,26/10/2012,46.97,3.755,38.65,2.61,0.98,NaN,457.74,5.217,0,26,10,2012
176,177,45,05/10/2012,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,8.667,0,5,10,2012
177,178,45,12/10/2012,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,8.667,0,12,10,2012
178,179,45,19/10/2012,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,8.667,0,19,10,2012


In [3]:
def splitStore(train, test, i):
    train = train[train.Store == i]
    test = test[test.Store == i]
    
    train = train.reset_index()
    train1 = train.iloc[100:]
    vali = train.iloc[88:91]

    return train1, test, vali


def reArrange(train, test, vali):
    xtrain = train[['Temperature', 'Fuel_Price','Promotion3', 'month']]
    ytrain = train[["Weekly_Sales"]]
    xvali = vali[['Temperature', 'Fuel_Price','Promotion3', 'month']]
    yvali = vali[["Weekly_Sales"]]
    xtest = test[['Temperature', 'Fuel_Price', 'Promotion3', 'month']]
    
    return xtrain, ytrain, xtest, xvali, yvali

def scaling(xtrain, xtest, xvali):
    scaler = MinMaxScaler()
    xtrain = pd.DataFrame(scaler.fit_transform(xtrain))
    xtest = pd.DataFrame(scaler.fit_transform(xtest))
    xvali = pd.DataFrame(scaler.fit_transform(xvali))
    
    return xtrain, xtest, xvali

In [4]:
def xgbModel(xtrain, ytrain, xtest, xvali, yvali):
    eval_set = [(xvali, yvali)]
    xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, subsample=0.7,
                           colsample_bytree=0.7, max_depth=5, silent = 1, nthread = 4, min_child_weight = 4)
    
    xgb_model.fit(xtrain,ytrain, eval_set=eval_set, eval_metric="rmse", early_stopping_rounds=20)
    pred = xgb_model.predict(xtest)
    
    return pred

In [5]:
def splitStore2(train, test, i):
    train = train[train.Store == i]
    test = test[test.Store == i]

    train = train.reset_index()
    train1 = train.iloc[100:]
    
    vali = train[train.month == 10]

    return train1, test, vali


def reArrange2(train, test, vali):
    xtrain = train[['Temperature', 'Fuel_Price', 'Unemployment', 'IsHoliday','month']]
    ytrain = train[["Weekly_Sales"]]
    xtest = test[['Temperature', 'Fuel_Price', 'Unemployment' ,'IsHoliday', 'month']]
    xvali = vali[['Temperature', 'Fuel_Price', 'Unemployment', 'IsHoliday','month']]
    yvali = vali[["Weekly_Sales"]]

    return xtrain, ytrain, xtest, xvali, yvali

def scaling2(xtrain, xtest, xvali):
    scaler = MinMaxScaler()
    xtrain = pd.DataFrame(scaler.fit_transform(xtrain))
    xtest = pd.DataFrame(scaler.fit_transform(xtest))
    xvali = pd.DataFrame(scaler.fit_transform(xvali))
    
    return xtrain, xtest, xvali

In [6]:
def randomForestModel(xtrain, ytrain, xtest):
    model = RandomForestRegressor()
    model.fit(xtrain,ytrain)
    pred = model.predict(xtest)
    
    return pred

In [7]:
result = []
exc = [36, 38 , 42, 43, 44] ## 비 선형 지점
for i in range(1, 46):
    if i in exc:
        print(i)
        train1, test1, vali = splitStore2(train, test, i)
        xtrain, ytrain, xtest, xvali, yvali = reArrange2(train1, test1, vali)
        xtrain, xtest, xvali = scaling2(xtrain, xtest, xvali)
        predict = randomForestModel(xtrain, ytrain, xtest)
    else:
        print(i)
        train1, test1, vali = splitStore(train, test, i)
        xtrain, ytrain, xtest, xvali, yvali = reArrange(train1, test1, vali)
        xtrain, xtest, xvali = scaling(xtrain, xtest, xvali)
        predict = xgbModel(xtrain, ytrain, xtest, xvali, yvali)

    for i in predict:
        result.append(i)

1
[21:19:42] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:1405283.03386
[1]	validation_0-rmse:1331071.35378
[2]	validation_0-rmse:1261858.40813
[3]	validation_0-rmse:1195908.10317
[4]	validation_0-rmse:1133313.10835
[5]	validation_0-rmse:1072681.76975
[6]	validation_0-rmse:1015564.36863
[7]	validation_0-rmse:962067.33491
[8]	validation_0-rmse:909870.59119
[9]	validation_0-rmse:860434.15314
[10]	validation_0-rmse:813823.79868
[11]	validation_0-rmse:769251.85084
[12]	validation_0-rmse:726618.70235
[13]	validation_0-rmse:685340.14635
[14]	validation_0-rmse:647930.53810
[15]	validation

/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: Runtime

[17]	validation_0-rmse:667253.00381
[18]	validation_0-rmse:629337.70123
[19]	validation_0-rmse:593156.81540
[20]	validation_0-rmse:559330.12765
[21]	validation_0-rmse:526682.73922
[22]	validation_0-rmse:497188.71131
[23]	validation_0-rmse:468285.69231
[24]	validation_0-rmse:440030.39158
[25]	validation_0-rmse:424691.78363
[26]	validation_0-rmse:400578.74207
[27]	validation_0-rmse:375997.36918
[28]	validation_0-rmse:353005.37345
[29]	validation_0-rmse:329727.54037
[30]	validation_0-rmse:308691.85312
[31]	validation_0-rmse:288721.40995
[32]	validation_0-rmse:269816.57169
[33]	validation_0-rmse:261357.57297
[34]	validation_0-rmse:243830.02464
[35]	validation_0-rmse:228045.59413
[36]	validation_0-rmse:212703.06203
[37]	validation_0-rmse:198537.97009
[38]	validation_0-rmse:185622.81975
[39]	validation_0-rmse:173468.45720
[40]	validation_0-rmse:160636.32824
[41]	validation_0-rmse:149319.31287
[42]	validation_0-rmse:138287.77446
[43]	validation_0-rmse:127970.96471
[44]	validation_0-rmse:12396

/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: Runtime

[51]	validation_0-rmse:170486.93870
[52]	validation_0-rmse:162732.59709
[53]	validation_0-rmse:155786.46418
[54]	validation_0-rmse:148792.25786
[55]	validation_0-rmse:141744.10144
[56]	validation_0-rmse:134299.39536
[57]	validation_0-rmse:131718.14628
[58]	validation_0-rmse:125321.96556
[59]	validation_0-rmse:120050.82941
[60]	validation_0-rmse:118610.57779
[61]	validation_0-rmse:114159.72081
[62]	validation_0-rmse:110072.07801
[63]	validation_0-rmse:108819.13165
[64]	validation_0-rmse:103152.43183
[65]	validation_0-rmse:104589.61274
[66]	validation_0-rmse:100412.13207
[67]	validation_0-rmse:95370.30369
[68]	validation_0-rmse:91501.41746
[69]	validation_0-rmse:93190.49016
[70]	validation_0-rmse:90014.11516
[71]	validation_0-rmse:89186.85966
[72]	validation_0-rmse:85298.51581
[73]	validation_0-rmse:82260.12905
[74]	validation_0-rmse:77449.81412
[75]	validation_0-rmse:73510.72999
[76]	validation_0-rmse:69972.57901
[77]	validation_0-rmse:69924.29338
[78]	validation_0-rmse:67416.71464
[79]

...

/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[122]	validation_0-rmse:54003.96647
[123]	validation_0-rmse:53580.10845
[124]	validation_0-rmse:53025.57551
[125]	validation_0-rmse:53216.01513
[126]	validation_0-rmse:53371.97687
[127]	validation_0-rmse:53234.78993
[128]	validation_0-rmse:53273.47788
[129]	validation_0-rmse:53244.42175
[130]	validation_0-rmse:53197.13576
[131]	validation_0-rmse:53319.38044
[132]	validation_0-rmse:53256.23053
[133]	validation_0-rmse:53481.25144
[134]	validation_0-rmse:53068.75158
[135]	validation_0-rmse:53267.38489
[136]	validation_0-rmse:53424.92951
[137]	validation_0-rmse:53050.94789
[138]	validation_0-rmse:53334.87414
[139]	validation_0-rmse:53324.00275
[140]	validation_0-rmse:53521.06407
[141]	validation_0-rmse:53773.72166
[142]	validation_0-rmse:53943.57487
[143]	validation_0-rmse:54127.97379
42
43
44


/var/folders/sn/xcqdgbg964vbr2j1_2_85gfc0000gn/T/ipykernel_69711/3603891260.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(xtrain,ytrain)
/var/folders/sn/xcqdgbg964vbr2j1_2_85gfc0000gn/T/ipykernel_69711/3603891260.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(xtrain,ytrain)
/var/folders/sn/xcqdgbg964vbr2j1_2_85gfc0000gn/T/ipykernel_69711/3603891260.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(xtrain,ytrain)


45
[21:19:51] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:722200.85695
[1]	validation_0-rmse:686675.57831
[2]	validation_0-rmse:653313.91431
[3]	validation_0-rmse:621493.60126
[4]	validation_0-rmse:591310.94053
[5]	validation_0-rmse:562296.64279
[6]	validation_0-rmse:535111.52313
[7]	validation_0-rmse:508927.71563
[8]	validation_0-rmse:483816.64000
[9]	validation_0-rmse:460117.30762
[10]	validation_0-rmse:437682.15760
[11]	validation_0-rmse:416108.59547
[12]	validation_0-rmse:395457.17905
[13]	validation_0-rmse:375748.93632
[14]	validation_0-rmse:357977.84280
[15]	validation_0-rms

/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Users/heo/opt/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


In [9]:
submission = pd.read_csv('./data/sample_submission.csv')

submission['Weekly_Sales'] = result
submission.to_csv("./data/submission_220804.csv", index = False)

submission

,id,Weekly_Sales
0,1,1.533239e+06
1,2,1.535430e+06
2,3,1.467244e+06
3,4,1.512324e+06
4,5,1.792166e+06
...,...,...
175,176,3.156412e+05
176,177,7.335580e+05
177,178,7.163788e+05
178,179,7.170746e+05
